In [123]:
import pyscf
import pyscf.tools

In [124]:
molecule = """
H 0.0 0.0 0.0
H 2.0 0.0 0.0
H 4.0 0.0 0.0
He 0.0 2.0 0.0
He 2.0 2.0 0.0
He 4.0 2.0 0.0
"""

In [125]:
basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   0, # number of unpaired electrons
        charge  =   1,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"


symmetry:  C2v


In [126]:

mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
# mf.analyze()



******** <class 'pyscf.scf.hf_symm.SymAdaptedROHF'> ********
method = SymAdaptedROHF-ROHF-RHF
initial guess = sad
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 0 MB)
num. doubly occ = 4  num. singly occ = 0
init E= -9.59006737893433
HOMO (A1) = -0.271010029950973  LUMO (B1) = -0.174829424666389
cycle= 1 E= -9.55065996713398  delta_E= 0.0394  |g|= 0.0971  |ddm|= 0.706
HOMO (A1) = -0.640579004468074  LUMO (B1) = -0.388545403180567
cycle= 2 E= -9.55413802628591  delta_E= -0.00348  |g|= 0.0144  |ddm|= 0.0907
HOMO (A1) = -0.634454764428176  LUMO (B1) = -0.382589401618681
cycle= 3 E= -9.55423338261332  delta_E= -9.54e-05  |g|= 0.00104  |ddm|= 0.0199
HOMO (A1) = -0.634546076728867  LUMO (B1) = -0.382418234191235
cycle= 4 E= -9.55423412891879  del

In [127]:
F = mf.get_fock()

In [128]:
import numpy as np
import scipy
import copy as cp
import math


def sym_ortho(frags, S, thresh=1e-8):
    """
    frags is a list of mo-coeff matrices
    """
    Nbas = S.shape[1]
    
    inds = []
    Cnonorth = np.hstack(frags)
    shift = 0
    for f in frags:
        inds.append(list(range(shift, shift+f.shape[1])))
        shift += f.shape[1]
        
    
    Smo = Cnonorth.T @ S @ Cnonorth
    X = np.linalg.inv(scipy.linalg.sqrtm(Smo))
    # print(Cnonorth.shape, X.shape)
    Corth = Cnonorth @ X
    
    frags2 = []
    for f in inds:
        frags2.append(Corth[:,f])
    return frags2


In [129]:
import scipy

# Find AO's corresponding to atoms 
# print(mf.mol.aoslice_by_atom())
# print(mf.mol.ao_labels(fmt=False, base=0))
full = []
frag1 = []
frag2 = []
frag3 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] in (0, 1, 2):
        if ao[2] in ("1s",):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 3:
        if ao[2] in ("1s", ):
            frag2.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 4:
        if ao[2] in ("1s", ):
            frag3.append(ao_idx)
            full.append(ao_idx)


frags = [frag1, frag2, frag3]
print(frags)



[[0, 5, 10], [15], [20]]


In [130]:
def svd_orbs(C, frag, S, thresh=1e-7, verbose=2):
    print(" Frag: ", frag)
    X = scipy.linalg.sqrtm(S)
    Cfrag = X@C
    
    _,s,V = np.linalg.svd(Cfrag[frag,:], full_matrices=True)
    print("Singular Values")
    for i in range(len(s)):
        print(" %4i : %12.8f" %(i, s[i]))
    return C @ V.T

C = mf.mo_coeff
S = mf.get_ovlp()
ndocc = mf.nelec[1]
nsing = mf.nelec[0] - ndocc
nvirt = mf.mol.nao - ndocc - nsing
# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]
print(ndocc, nsing, nvirt)
print(Cdocc.shape)
print(Csing.shape)
print(Cvirt.shape)

print(" Doubly Occupied")
Cdocc = svd_orbs(Cdocc, full, S)
print(" Doubly Unoccupied")
Cvirt = svd_orbs(Cvirt, full, S)

# Ctmp = rmv_svd_orbs(Csing, full, S)

pyscf.tools.molden.from_mo(mf.mol, "Cdocc.molden", Cdocc)
pyscf.tools.molden.from_mo(mf.mol, "Cvirt.molden", Cvirt)
# pyscf.tools.molden.from_mo(mf.mol, "Ctmp.molden", Ctmp)

4 0 26
(30, 4)
(30, 0)
(30, 26)
 Doubly Occupied
 Frag:  [0, 5, 10, 15, 20]
Singular Values
    0 :   0.74675406
    1 :   0.72695476
    2 :   0.63653102
    3 :   0.02574556
 Doubly Unoccupied
 Frag:  [0, 5, 10, 15, 20]
Singular Values
    0 :   1.00000000
    1 :   0.99966853
    2 :   0.77125110
    3 :   0.68668535
    4 :   0.66510028

WARN: orbitals [0 1 2 3] not symmetrized, norm = [0.82026767 0.70500688 0.97632373 0.50159828]


WARN: orbitals [ 0  1  2  3  4  5  6  7  8 11 12 13 15 16 17 21 22 24 25] not symmetrized, norm = [0.69556154 0.6954474  0.98576044 0.70819559 0.80001774 0.51967413
 0.8244385  0.99738305 0.98006682 0.907534   0.91281353 0.91888369
 0.93857589 0.9989632  0.9896431  0.98332105 0.99822076 0.99006086
 0.98163334]



In [172]:
import spade_clustering
from spade_clustering import *

nbas = Cdocc.shape[0]

P1 = np.eye(nbas)[:,frag1] 
P2 = np.eye(nbas)[:,frag2] 
P3 = np.eye(nbas)[:,frag3] 

O1, V1 = spade_partitioning(Cdocc, Cvirt, P1, S)
O2, V2 = spade_partitioning(Cdocc, Cvirt, P2, S)
O3, V3 = spade_partitioning(Cdocc, Cvirt, P3, S)

pyscf.tools.molden.from_mo(mf.mol, "CO1.molden", O1)
pyscf.tools.molden.from_mo(mf.mol, "CO2.molden", O2)
pyscf.tools.molden.from_mo(mf.mol, "CV1.molden", V1)
pyscf.tools.molden.from_mo(mf.mol, "CV2.molden", V2)


NameError: name 'spade_partitioning' is not defined

In [ ]:


# nact_docc = 1
# nact_virt = 1

# nfrzn = Cdocc.shape[1] - nact_docc
# Cenv = Cdocc[:,nact_docc:nact_docc+nfrzn]
# Cact = np.hstack((Cdocc[:,0:nact_docc], Csing, Cvirt[:,0:nact_virt]))
# print(Cenv.shape)
# print(Cact.shape)
# print("Should be zero: ", np.trace(Cenv.T@S@Cact@Cact.T@Cenv))

# Cfrag1 = svd_orbs(Cact, frag1, S)[:,0:3]
# Cfrag2 = svd_orbs(Cact, frag2, S)[:,0:1]
# Cfrag3 = svd_orbs(Cact, frag3, S)[:,0:1]

# Cfrag1, Cfrag2, Cfrag3 = sym_ortho((Cfrag1, Cfrag2, Cfrag3), S)

# pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)

# pyscf.tools.molden.from_mo(mf.mol, "Cfrag1.molden", Cfrag1)
# pyscf.tools.molden.from_mo(mf.mol, "Cfrag2.molden", Cfrag2)
# pyscf.tools.molden.from_mo(mf.mol, "Cfrag3.molden", Cfrag3)

# Ctot = np.hstack((Cfrag1, Cfrag2, Cfrag3))
# pyscf.tools.molden.from_mo(mf.mol, "Ctot.molden", Ctot)
# pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)


In [132]:
clusters = [list(range(0,6)), list(range(6,10)), list(range(10,16))]
init_fspace = [(3,0), (4,4), (3,0)]
print(clusters)
print(init_fspace)

[[0, 1, 2, 3, 4, 5], [6, 7, 8, 9], [10, 11, 12, 13, 14, 15]]
[(3, 0), (4, 4), (3, 0)]


# Make Integrals

In [133]:
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

In [134]:
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Ctot.T @ h @ Ctot;
j = Ctot.T @ j @ Ctot;
k = Ctot.T @ k @ Ctot;

In [135]:
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Ctot, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

NotImplementedError: Integral transformation for complex orbitals

In [ ]:
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Ctot @ Ctot.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;


In [ ]:
np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Ctot)
np.save("overlap_mat", S)

In [ ]:
import numpy as np
Ccmf = np.load("Ccmf.npy")
pyscf.tools.molden.from_mo(mf.mol, "Ccmf.molden", Ccmf)



WARN: orbitals [3 4] not symmetrized, norm = [0.76812236 0.76812236]



In [ ]:
print(Cdocc.shape)
print(Csing.shape)
P = 2*Cdocc@Cdocc.T + Csing@Csing.T
print(np.trace(P@S))
print(Ctot.shape)


(30, 4)
(30, 0)
8.0
(30, 4)


In [ ]:
Ptot = Ctot @ Ctot.T
Pact = Cact @ Cact.T 
Penv = Cenv @ Cenv.T
Psing = Csing @ Csing.T
Pdocc = Cdocc @ Cdocc.T
Pvirt = Cvirt @ Cvirt.T
print(Cenv.shape)

(30, 3)
